# Bubble sort

In [61]:
def lprint(A,start=-1,stop=-1):
    for i,a in enumerate(A):
        print(f"{a:3}", end=' ')
        print("\b", end="")
        if i>=start and i<=stop: print("\u0332", end="")
        
def bubble(A):
    swapped=True
    second_to_last_idx = len(A)-2
    n = 1
    while swapped:
        swapped=False
        print(); lprint(A); print(f"\t\tPass {n}"); n += 1
        for i in range(second_to_last_idx+1):
            if A[i] > A[i+1]:
                A[i], A[i+1] = A[i+1], A[i]
                swapped = True
                lprint(A,i,i+1); print(f"\t\tSwap A[{i}], A[{i+1}]")

In [60]:
A = [9,1,4,2,-3]
bubble(A)


  9  1  4  2 -3		Pass 1
  1̲  9̲  4  2 -3		Swap A[0], A[1]
  1  4̲  9̲  2 -3		Swap A[1], A[2]
  1  4  2̲  9̲ -3		Swap A[2], A[3]
  1  4  2 -3̲  9̲		Swap A[3], A[4]

  1  4  2 -3  9		Pass 2
  1  2̲  4̲ -3  9		Swap A[1], A[2]
  1  2 -3̲  4̲  9		Swap A[2], A[3]

  1  2 -3  4  9		Pass 3
  1 -3̲  2̲  4  9		Swap A[1], A[2]

  1 -3  2  4  9		Pass 4
 -3̲  1̲  2  4  9		Swap A[0], A[1]

 -3  1  2  4  9		Pass 5


# Quicksort

In [57]:
def partition(A,lo,hi):
    pivot = A[hi]    # pick last element as pivot  
    left = [a for a in A if a<pivot]
    right = [a for a in A if a>pivot]
    A[:] = left+[pivot]+right # copy back to A
    return len(left) # return index of pivot

def partition_inplace(A,lo,hi):
    """
    Wikipedia: reorder the array so that all elements with
    values less than the pivot come before the pivot, while
    all elements with values greater than the pivot come after
    it (equal values can go either way).
    """
    i = lo
    pivot = A[hi]    # pick last element as pivot  
    for j in range(lo, hi):  
        if A[j] < pivot:   
            A[i], A[j] = A[j], A[i]
            i += 1  
    A[i], A[hi] = A[hi], A[i]
    return i

In [58]:
def qsort(A, lo=0, hi=len(A)-1):
    if lo >= hi: return
    pivot_idx = partition(A,lo,hi)
    qsort(A, lo, pivot_idx-1)
    qsort(A, pivot_idx+1, hi)

In [59]:
A = [9,1,4,2,-3]
qsort(A)
print(A)

[-3, 1, 2, 4, 9]


# Pigeonhole sort

In [10]:
# avoid negative numbers for simplicity
# also assume min(A) is close to 0
def psort(A:list) -> list:
    size = max(A) + 1
    holes = [0] * size
    for a in A:
        holes[a] += 1
    A_ = []
    for i in range(0,size):
        for j in range(holes[i]):
            A_.append(i)
    return A_

A = [9,1,4,2,8,2,9] 
psort(A)

[1, 2, 2, 4, 8, 9, 9]

# Bucket sort

In [36]:
import numpy as np

def bsort(A:list, nbuckets) -> list:
    mx = max(A)
    buckets = []
    max_bucket_idx = nbuckets+1
    for i in range(max_bucket_idx + 1):
        buckets.append([])
    for a in A:
        a_normalized = a / mx # get in 0..1
        i = int(a_normalized * nbuckets) # spread across buckets
        buckets[i].append(a)
        
    A_ = []
    for i in range(max_bucket_idx+1):
        A_.extend( sorted(buckets[i]) )
    return A_

A = np.random.random(size=10) * 100 + 10
bsort(A, 5)

[13.159916470990476,
 19.81124696697899,
 42.53983669943354,
 51.4750865088357,
 63.94121794311986,
 75.80755727501925,
 84.84191385167692,
 90.70348332761021,
 92.58559518237188,
 92.7766522367175]

# Bucket sort on strings

In [53]:
# assume lowercase English letters for simplicity
def pstr_sort(A:list) -> list:
    size = ord('z') - ord('a') + 1
    holes = []
    for i in range(size):
        holes.append([])
    for s in A:
        i = ord(s[0])-ord('a')
        holes[i].append(s)
    A_ = []
    for i in range(ord('z')-ord('a') + 1):
        A_.extend( sorted(holes[i]) )
    return A_

A = ['apple', 'ape', 'zebra', 'cat', 'canary', 'civet', 'dog'] 
pstr_sort(A)

['ape', 'apple', 'canary', 'cat', 'civet', 'dog', 'zebra']